In [1]:
import socket
import threading
import Classes as xClass
import random
import secrets

In [2]:
List_Users=[]
enc_list=[]
index=0
client_sockets = {}

In [3]:
# pnn=xClass.TTP_Algorithm
# pnn.p=pnn.generate_random_prime(pnn)
# pnn.q=pnn.generate_random_prime(pnn)
# pnn.alpha=pnn.alphaa(pnn.q)
# pnn.PrivateKey=pnn.Calc_xA_and_xB(pnn.q)
# pnn.PublicKey=pnn.Calc_yA_and_yB_PrivKey(pnn.alpha,pnn.PrivateKey,pnn.q)
# List_Users.append((pnn,('127.0.0.1', 36984)))


In [4]:
def integer_to_ascii_value(integer_value):
    # Convert the integer value to its corresponding ASCII character
    return chr(integer_value)

# Example usage:||
ascii_value = ord('b')  # Replace 'A' with the character you want to convert
print(ascii_value)
original_char = integer_to_ascii_value(ascii_value)
print(original_char)


98
b


In [5]:


#----------------------------------------------------------------
#RC4
def text_to_decimal_string(text):
    result = ""    
    for char in text:
        # Get the ASCII value of the character
        ascii_value = ord(char)       
        # Take the remainder when dividing by 5 to get a number between 0 and 4
        decimal_value = ascii_value % 5
        # Convert the decimal value to a string and append to the result
        result += str(decimal_value)

    # Split the result into chunks of four characters
    chunks = [result[i:i+4] for i in range(0, len(result), 4)]

    # Check if the last chunk has less than 4 characters and pad with 0
    if len(chunks[-1]) < 4:
        chunks[-1] = chunks[-1].ljust(4, '0')

    return chunks
#----------------------------------------------------------------
#ELGMAL
def text_to_decimal(text):
    decimal_values = [ord(char) for char in text]
    return decimal_values




In [6]:
def pad_hex_to_128_bits(hex_string):
    # Calculate the number of hex digits needed for padding
    padding_length = 32 - len(hex_string)

    # Pad the hex string with zeros to the left
    padded_hex = '0' * padding_length + hex_string

    return padded_hex

def remove_padding(hex_string):
    # Find the index of the first non-zero character
    index_of_non_zero = next((i for i, c in enumerate(hex_string) if c != '0'), None)

    # If all characters are zeros, keep one zero
    if index_of_non_zero is None:
        return '0'

    # Remove the leading zeros or padding characters
    unpadded_hex = hex_string[index_of_non_zero:]

    return unpadded_hex


def text_to_hex(input_text):
    # Encode the text into bytes
    byte_representation = input_text.encode('utf-8')

    # Convert the bytes to a hexadecimal string
    hex_representation = byte_representation.hex()

    return hex_representation


def hex_to_128bit(hex_input):
    # Pad the hexadecimal string with zeros to the left
    padded_hex = hex_input.zfill(32)

    return padded_hex

def hex_to_text(hex_string):
    try:
        # Convert hexadecimal string to bytes
        byte_data = bytes.fromhex(hex_string)
        
        # Decode bytes to text using UTF-8 encoding (you can change encoding if needed)
        text_data = byte_data.decode('utf-8')
        
        return text_data
    except ValueError:
        # Handle invalid hexadecimal string
        return "Invalid hexadecimal string"

def split_hex_into_chunks(hex_string, chunk_size):
    # Check if the length of the hexadecimal string is divisible by chunk_size
    remainder = len(hex_string) % chunk_size
    if remainder != 0:
        # Pad the hexadecimal string with zeros to make it divisible by chunk_size
        hex_string = hex_string + '0' * (chunk_size - remainder)

    # Split the padded hexadecimal string into chunks of the specified size
    chunks = [hex_string[i:i + chunk_size] for i in range(0, len(hex_string), chunk_size)]

    return chunks





#print(s.WKey)



Process

In [7]:
def AES(num):
    result=text_to_hex(num)    
    chunks=split_hex_into_chunks(result,16)
    KEY="0f1571c947d9e8590cb7add6af7f6798"
    Cypher_Message=[]
    Message=[]
    s=xClass.AES_Algorithm
    for value in chunks:
        plaintext=value
        Cypher_Message.append(s.Encrypt(s,KEY,plaintext))
    for value in Cypher_Message:
        Message.append(s.Decrypt(s,KEY,value))
    enc_list.append(("AES",''.join(Cypher_Message).lower())) 
    if Message==chunks:        
        return num[0:-1]
    return f"Error message Couldnt Decrypt Right: {''.join(Message)}"

def RC4(num):
    result = text_to_decimal_string(num)
    KEY="1236"
    Cypher_Message=[]
    Message=[]
    s=xClass.RC4_Algorithm
    for value in result:
        Cypher_Message.append(str(s.encrypt(s,str(value),KEY)))
    for value in Cypher_Message:
        Message.append(str(s.decrypt(s,str(value),KEY)))
    enc_list.append(("RC4",''.join(Cypher_Message).lower()) )
    if Message==result:        
        return num[0:-1]
    return "Error message"

def ELGAMAL(num):
    result=text_to_decimal(num)    
    s=xClass.ELGAMAL_Algorithm
    Cypher_Message=[]
    Message=[]   
    ssw="" 
    pnn=xClass.TTP_Algorithm
    pnn.p=pnn.generate_random_prime(pnn)
    pnn.q=pnn.generate_random_prime(pnn)
    pnn.alpha=pnn.alphaa(pnn.q)
    pnn.PrivateKey=pnn.Calc_xA_and_xB(pnn.q)
    pnn.PublicKey=pnn.Calc_yA_and_yB_PrivKey(pnn.alpha,pnn.PrivateKey,pnn.q)
    for value in result:
        Cypher_text,C2=s.encrypt(s,pnn,value)
        Cypher_Message.append((Cypher_text,C2))
    # print(Cypher_Message[0][0],Cypher_Message[0][1],Cypher_Message[0])
    for value in Cypher_Message:
        ssw+=str(value[0])
        Message.append(s.decrypt(pnn,value[0],value[1]))    
    enc_list.append(("ELGAMAL",ssw))         
    if  Message==result:
        return num[0:-1]
    return f"Error message Couldnt Decrypt Right:"

    

def RSA(num):
    result=text_to_decimal(num)
    ss=xClass.RSA_Algorithm
    Cypher_Message=[]
    Message=[] 
    pnn=xClass.TTP_Algorithm
    pnn.p=17
    pnn.q=11
    pnn.alpha=pnn.alphaa(pnn.q)
    pnn.PrivateKey=pnn.Calc_xA_and_xB(pnn.q)
    pnn.PublicKey=pnn.Calc_yA_and_yB_PrivKey(pnn.alpha,pnn.PrivateKey,pnn.q) 
    for value in result:
        Cypher_Message.append(ss.encrypt(ss,value,pnn))            
    for value in Cypher_Message:
        Message.append(ss.decrypt(value,ss.d,ss.n))      
    enc_list.append(("RSA",''.join(str(Cypher_Message)).lower()))
    if  Message==result:
        return num[0:-1]
    stt=str(Message)
    return f"Error message Couldnt Decrypt Right: {''.join(stt)}"

def DES(num):
    result=text_to_hex(num)    
    chunks=split_hex_into_chunks(result,16)
    KEY="0f1571c947d9e859" 
    Cypher_Message=[]
    Message=[]
    s=xClass.DES_Algorithm
    for value in chunks:
        Cypher_Message.append(str(s.bin_to_hexa(s.encrypt(s,str(value),KEY))))
    for value in Cypher_Message:
        Message.append(str(s.bin_to_hexa(s.decrypt(s,str(value),KEY))))   
    # print(''.join(Message).lower(),''.join(chunks).lower()) 

    enc_list.append(("DES",''.join(Cypher_Message).lower()))
    if ''.join(Message).lower()==''.join(chunks).lower():        
        return num[0:-1]
    return "Error message"

def Vigenere(num):
    KEY="mewo"
    s=xClass.Vigenere_Algorithm
    Cypher_Message=s.encrypt(s,num,KEY)
    Message=s.decrypt(s,Cypher_Message,KEY)
    enc_list.append(Cypher_Message)
    if Message==num:
        return num[0:-1]
    return "Error message"

def Row_Transposition(num):
    KEY="4321"
    s=xClass.Row_Transposition_Algorithm(KEY,num)
    Cypher_Message=s.encrypt()
    Message=s.decrypt()
    Message = Message[1].rstrip('Z').lower()   
    enc_list.append(("Row_Transposition",Cypher_Message)) 
    if Message==num:
        return num[0:-1]
    return "Error message"

def Monoalphabet(num):
    KEY="iamgoingfarr"
    s=xClass.Monoalphabet_Algorithm(KEY,num)
    Cypher_Message=s.perform_encryption()
    Message=s.perform_decryption(Cypher_Message)  
    enc_list.append(("Monoalphabet",Cypher_Message))
    if ''.join(Message).lower()==''.join(num).lower():
        return num[0:-1]
    return "Error message"

def Ceaser(num):
    KEY=3
    s=xClass.Caesar_Algorithm(num,KEY)
    Cypher_Message=s.encrypt()
    Message=s.decrypt(Cypher_Message)
    enc_list.append(("Ceaser",Cypher_Message))
    if Message[0:-1]==num[0:-1]:
        return num[0:-1]
    return "Error message"

def Playfair(num):
    key = "KEYWORD"    
    s = xClass.Playfair_Algorithm()            
    key_matrix_encryption = s.key_generation(key)
    result_encryption = s.encrypt(num, key_matrix_encryption)
    ciphertext_input = result_encryption
    key_matrix_decryption = s.key_generation(key)
    result_decryption = s.decryption(ciphertext_input, key_matrix_decryption)  
    enc_list.append(("Playfair",ciphertext_input))
    if result_decryption[0:-1]==num[0:-1]:
        return num[0:-1]
    return "Error message"

def RailFence(num):
    key = 3
    s=xClass.RailFence_Algorithm(key)
    Cypher_Message=s.encrypt(num)
    enc_list.append(("RailFence",Cypher_Message))
    Message=s.decrypt(Cypher_Message)
    if Message[0:-1]==num[0:-1]:
        return num[0:-1]
    return "Error message"

    
def Type_of_Algorithm(num):
    switch_dict = {
    '1': AES,
    '2': RC4,
    '3': ELGAMAL,
    '4': RSA,
    '5':DES,
    '6':Vigenere,
    '7':Row_Transposition,
    '8':Monoalphabet,
    '9':Ceaser,
    '0':Playfair,
    'x':RailFence
    }


    if num[len(num)-1] in switch_dict:
        result = str(switch_dict[num[-1]](num))
    with open("encrypt.txt", "w") as file:        
        for value in enc_list:
            file.write(f"{value}\n")
    return result


print(Type_of_Algorithm("ahmed4"))
# print(enc_list)


ahmed


In [8]:
def know_index(address):
    i=0
    for pnn, other_address in List_Users:
        if other_address != address:
            index=i
            break
        else:
            i+=1

Host

In [9]:




def handle_client(client_socket, address):
    print(f"Connection from {address}")
    pnn=xClass.TTP_Algorithm
    # pnn.generate_random_prime(pnn)
    pnn.p=17
    pnn.q=11
    pnn.alpha=pnn.alphaa(pnn.q)
    pnn.PrivateKey=pnn.Calc_xA_and_xB(pnn.q)
    pnn.PublicKey=pnn.Calc_yA_and_yB_PrivKey(pnn.alpha,pnn.PrivateKey,pnn.q)
    List_Users.append((pnn,address)) 
    # know_index(address)    
    # print(List_Users[0][0])   
    try:
        while True:
            data = client_socket.recv(1024).decode('utf-8')
            if not data:
                break
            for other_address, other_socket in client_sockets.items():
                if other_address != address:
                    try:                       
                        print(data)
                        data=Type_of_Algorithm(data)
                        other_socket.send(data.encode('utf-8'))
                    except:
                        # Remove the socket if it is no longer valid
                        del client_sockets[other_address]

    except Exception as e:
        print(f"Error in handle_client: {e}")

    finally:
        print(f"Connection from {address} closed")
        # Remove the client socket from the dictionary
        del client_sockets[address]
        client_socket.close()

def start_server():
    host = 'localhost'  # Server IP address
    port = 5000       # Port to bind

    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(5)

    print(f"Server listening on {host}:{port}")

    while True:
        client_socket, address = server_socket.accept()
        client_sockets[address] = client_socket
        client_handler = threading.Thread(target=handle_client, args=(client_socket, address))
        client_handler.start()

if __name__ == "__main__":
    start_server()


Server listening on localhost:5000
Connection from ('127.0.0.1', 5353)
Connection from ('127.0.0.1', 5357)
dsad0
dw1
dawq2
wd3
ahmed0
wdadas4
mewo5
kjhfdjkgh6
tyhrh7
reg8
fe8
ergeg9
m3rfsh2


In [ ]:
# # Example usage
# playfair_cipher = PlayfairCipher()

# # Encryption Example
# key = "KEYWORD"
# plaintext_input = "HELLO WORLD"
# key_matrix_encryption = playfair_cipher.key_generation(key)

# result_encryption = playfair_cipher.encrypt(plaintext_input, key_matrix_encryption)
# print("Result of Encryption:", result_encryption)


# ciphertext_input = result_encryption
# key_matrix_decryption = playfair_cipher.key_generation(key)


# result_decryption = playfair_cipher.decryption(ciphertext_input, key_matrix_decryption)
# print("Result of Decryption:", result_decryption)

Result of Encryption: gyizscokcfbu
Result of Decryption: helxloworldx
